In [2]:
pip freeze > model_requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

In [4]:
data_mm_train_df = pd.read_csv('data_smin_mm_train_csv')
data_mm_test_df = pd.read_csv('data_mm_test_csv')
data_mm_val_df = pd.read_csv('data_mm_val_csv')

data_mm_train_df = data_mm_train_df.drop(['Unnamed: 0', 'Remaining overs and balls', 'Remaining Team Value', 'Remaining Average Team Value'], axis=1)
data_mm_test_df = data_mm_test_df.drop(['Unnamed: 0', 'Remaining overs and balls', 'Remaining Team Value', 'Remaining Average Team Value'], axis=1)
data_mm_val_df = data_mm_val_df.drop(['Unnamed: 0', 'Remaining overs and balls', 'Remaining Team Value', 'Remaining Average Team Value'], axis=1)

In [ ]:
data_ss_train_df = pd.read_csv('data_smin_ss_train_csv')
data_ss_test_df = pd.read_csv('data_ss_test_csv')
data_ss_val_df = pd.read_csv('data_ss_val_csv')

data_ss_train_df = data_ss_train_df.drop(['Unnamed: 0', 'Remaining overs and balls', 'Remaining Team Value', 'Remaining Average Team Value'], axis=1)
data_ss_test_df = data_ss_test_df.drop(['Unnamed: 0', 'Remaining overs and balls', 'Remaining Team Value', 'Remaining Average Team Value'], axis=1)
data_ss_val_df = data_ss_val_df.drop(['Unnamed: 0', 'Remaining overs and balls', 'Remaining Team Value', 'Remaining Average Team Value'], axis=1)

In [ ]:
data_log_train_df = pd.read_csv('data_smin_log_train_csv')
data_log_test_df = pd.read_csv('data_test_csv')
data_log_val_df = pd.read_csv('data_val_csv')

data_log_train_df = data_log_train_df.drop(['Unnamed: 0', 'Remaining overs and balls', 'Remaining Team Value', 'Remaining Average Team Value'], axis=1)
data_log_test_df = data_log_test_df.drop(['Unnamed: 0', 'Remaining overs and balls', 'Remaining Team Value', 'Remaining Average Team Value'], axis=1)
data_log_val_df = data_log_val_df.drop(['Unnamed: 0', 'Remaining overs and balls', 'Remaining Team Value', 'Remaining Average Team Value'], axis=1)

In [ ]:
print(f"Data_train: {data_mm_train_df.shape}")
print(f"Data_test: {data_mm_test_df.shape}")
print(f"Data_val: {data_mm_val_df.shape}")

#Divide by 10 for the sample

In [8]:
data_mm_train_sample_df = data_mm_train_df.sample(n=28804, replace=False, random_state=7, axis=0)
data_mm_val_sample_df = data_mm_val_df.sample(n=2702, replace=False, random_state=7, axis=0)
data_mm_test_sample_df = data_mm_test_df.sample(n=5405, replace=False, random_state=7, axis=0)

data_ss_train_sample_df = data_ss_train_df.sample(n=28804, replace=False, random_state=7, axis=0)
data_ss_val_sample_df = data_ss_val_df.sample(n=2702, replace=False, random_state=7, axis=0)
data_ss_test_sample_df = data_ss_test_df.sample(n=5405, replace=False, random_state=7, axis=0)

In [11]:
y_mm_train_sample = data_mm_train_sample_df['Winner_num']
X_mm_train_sample = data_mm_train_sample_df.drop('Winner_num',axis=1)

X_mm_test_sample = data_mm_test_sample_df.drop('Winner_num',axis=1) 
y_mm_test_sample = data_mm_test_sample_df['Winner_num']

X_mm_val_sample = data_mm_val_sample_df.drop('Winner_num',axis=1) 
y_mm_val_sample = data_mm_val_sample_df['Winner_num']

y_ss_train_sample = data_ss_train_sample_df['Winner_num']
X_ss_train_sample = data_ss_train_sample_df.drop('Winner_num',axis=1)

X_ss_test_sample = data_mm_test_sample_df.drop('Winner_num',axis=1) 
y_ss_test_sample = data_mm_test_sample_df['Winner_num']

X_ss_val_sample = data_ss_val_sample_df.drop('Winner_num',axis=1) 
y_ss_val_sample = data_ss_val_sample_df['Winner_num']


y_log_train = data_log_train_df['Winner_num']
X_log_train = data_log_train_df.drop('Winner_num',axis=1)

X_log_test = data_log_test_df.drop('Winner_num',axis=1) 
y_log_test = data_log_test_df['Winner_num']

X_log_val = data_log_test_df.drop('Winner_num',axis=1) 
y_log_val = data_log_test_df['Winner_num']

SVM model

In [14]:
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

In [15]:
adfasdf

NameError: name 'adfasdf' is not defined

Baseline one SVC

In [ ]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel="linear", random_state=7)

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5992154540744579


In [ ]:
sdgdfg

NameError: name 'sdgdfg' is not defined

In [ ]:
k_list = ['linear', 'rbf', 'sigmoid']
gamma_list = [0.001, 0.01, 0.1, 1, 10, 100,200, 300]
c_list = [0.001, 0.01, 0.1, 1, 10, 100, 200, 300]

In [ ]:
print("MinMax")
best_accuracy = 0

for k in k_list:
    for gamma in gamma_list:
        for c in c_list:
        # for each combination of parameters, train an SVC
                
            model = svm.SVC(kernel=k, gamma=gamma, C=c, random_state = 7)
            model.fit(X_mm_train_sample, y_mm_train_sample)
            y_mm_pred = model.predict(X_mm_val_sample)
            accuracy = accuracy_score(y_mm_val_sample, y_mm_pred)
            print(f"{k}, Gamma: {gamma}, C: {c}, Accuracy: {accuracy}")

        # if we got a better score, store the score and parameters
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_parameters = {'C': c, 'gamma': gamma, 'kernel': k}

print("Best score: {:.2f}".format(best_accuracy))
print("Best parameters: {}".format(best_parameters))

linear, Gamma: 0.001, C: 0.001, Accuracy: 0.01
linear, Gamma: 0.001, C: 0.01, Accuracy: 0.5612
linear, Gamma: 0.001, C: 0.1, Accuracy: 0.5944
linear, Gamma: 0.001, C: 1, Accuracy: 0.5908
linear, Gamma: 0.001, C: 10, Accuracy: 0.5924
linear, Gamma: 0.001, C: 100, Accuracy: 0.5904
linear, Gamma: 0.001, C: 200, Accuracy: 0.5904
linear, Gamma: 0.001, C: 300, Accuracy: 0.5904
linear, Gamma: 0.01, C: 0.001, Accuracy: 0.01
linear, Gamma: 0.01, C: 0.01, Accuracy: 0.5612
linear, Gamma: 0.01, C: 0.1, Accuracy: 0.5944
linear, Gamma: 0.01, C: 1, Accuracy: 0.5908
linear, Gamma: 0.01, C: 10, Accuracy: 0.5924
linear, Gamma: 0.01, C: 100, Accuracy: 0.5904
linear, Gamma: 0.01, C: 200, Accuracy: 0.5904
linear, Gamma: 0.01, C: 300, Accuracy: 0.5904
linear, Gamma: 0.1, C: 0.001, Accuracy: 0.01
linear, Gamma: 0.1, C: 0.01, Accuracy: 0.5612
linear, Gamma: 0.1, C: 0.1, Accuracy: 0.5944
linear, Gamma: 0.1, C: 1, Accuracy: 0.5908
linear, Gamma: 0.1, C: 10, Accuracy: 0.5924
linear, Gamma: 0.1, C: 100, Accuracy:

In [ ]:
print("SS")
best_accuracy = 0

for k in k_list:
    for gamma in gamma_list:
        for c in c_list:
        # for each combination of parameters, train an SVC
                
            model = svm.SVC(kernel=k, gamma=gamma, C=c, random_state = 7)
            model.fit(X_ss_train_sample, y_ss_train_sample)
            y_ss_pred = model.predict(X_ss_val_sample)
            accuracy = accuracy_score(y_ss_val_sample, y_ss_pred)
            print(f"{k}, Gamma: {gamma}, C: {c}, Accuracy: {accuracy}")

        # if we got a better score, store the score and parameters
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_parameters = {'C': c, 'gamma': gamma, 'kernel': k}

print("Best score: {:.2f}".format(best_accuracy))
print("Best parameters: {}".format(best_parameters))

MinMax


NameError: name 'k_list' is not defined

In [ ]:
print("LogNorm")
best_accuracy = 0

for k in k_list:
    for gamma in gamma_list:
        for c in c_list:
        # for each combination of parameters, train an SVC
                
            model = svm.SVC(kernel=k, gamma=gamma, C=c, random_state = 7)
            model.fit(X_log_train_sample, y_log_train_sample)
            y_log_pred = model.predict(X_log_val_sample)
            accuracy = accuracy_score(y_log_val_sample, y_log_pred)
            print(f"{k}, Gamma: {gamma}, C: {c}, Accuracy: {accuracy}")

        # if we got a better score, store the score and parameters
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_parameters = {'C': c, 'gamma': gamma, 'kernel': k}

print("Best score: {:.2f}".format(best_accuracy))
print("Best parameters: {}".format(best_parameters))

rbf, Gamma: 100, C: 0.1, Accuracy: 0.6748
rbf, Gamma: 100, C: 0.2, Accuracy: 0.694
rbf, Gamma: 100, C: 0.3, Accuracy: 0.6972
rbf, Gamma: 100, C: 0.4, Accuracy: 0.7028
rbf, Gamma: 100, C: 0.5, Accuracy: 0.7104
rbf, Gamma: 100, C: 0.6, Accuracy: 0.7168
rbf, Gamma: 100, C: 0.7, Accuracy: 0.7144
rbf, Gamma: 100, C: 0.8, Accuracy: 0.714
rbf, Gamma: 100, C: 0.9, Accuracy: 0.7176
rbf, Gamma: 200, C: 0.1, Accuracy: 0.592
rbf, Gamma: 200, C: 0.2, Accuracy: 0.6776
rbf, Gamma: 200, C: 0.3, Accuracy: 0.7044
rbf, Gamma: 200, C: 0.4, Accuracy: 0.712
rbf, Gamma: 200, C: 0.5, Accuracy: 0.716
rbf, Gamma: 200, C: 0.6, Accuracy: 0.7176
rbf, Gamma: 200, C: 0.7, Accuracy: 0.7208
rbf, Gamma: 200, C: 0.8, Accuracy: 0.726
rbf, Gamma: 200, C: 0.9, Accuracy: 0.7248
rbf, Gamma: 300, C: 0.1, Accuracy: 0.5164
rbf, Gamma: 300, C: 0.2, Accuracy: 0.6164
rbf, Gamma: 300, C: 0.3, Accuracy: 0.6776
rbf, Gamma: 300, C: 0.4, Accuracy: 0.6956
rbf, Gamma: 300, C: 0.5, Accuracy: 0.7068
rbf, Gamma: 300, C: 0.6, Accuracy: 0.712

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define the SVM base classifier
svm_classifier = SVC()

# Define the parameters to tune (you can customize this based on your needs)
param_grid = {
    'base_estimator__C': [0.1, 1, 10],
    'base_estimator__kernel': ['linear', 'rbf', 'poly'],
    'base_estimator__gamma': ['scale', 'auto'],
    'n_estimators': [10, 50, 100],
}

# Create the bagging classifier with SVM as the base estimator
bagging_classifier = BaggingClassifier(base_estimator=svm_classifier, random_state=42)

# Create the GridSearchCV object for hyperparameter tuning
grid_search = GridSearchCV(bagging_classifier, param_grid, cv=3, scoring='accuracy')

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_sample, y_train_sample)

# Get the best parameters from the grid search
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Use the best parameters to create the final bagging classifier
final_bagging_classifier = BaggingClassifier(base_estimator=SVC(**best_params['base_estimator']), 
                                             n_estimators=best_params['n_estimators'],
                                             random_state=42)

# Train the final bagging classifier on the training data
final_bagging_classifier.fit(X_train_sample, y_train_sample)

# Evaluate the classifier on the validation set
val_predictions = final_bagging_classifier.predict(X_val)
val_accuracy = accuracy_score(y_val_sample, val_predictions)
print("Validation Accuracy:", val_accuracy)



# Evaluate the classifier on the test set
#test_predictions = final_bagging_classifier.predict(X_test)
#test_accuracy = accuracy_score(y_test, test_predictions)
#print("Test Accuracy:", test_accuracy)


C:\Users\carth\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\carth\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\carth\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\carth\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-pac

https://datascience.stackexchange.com/questions/66216/gridsearch-without-cv